## Analysis problem
The dataset that we want to analyze in this project is about number of deaths for viral hepatitis and sequelae of viral hepatitis in 25 European countries. Our dataset is divided into two files:
 - one contains the total amount of deaths for all causes of death from 2001 to 2010
 - the other contains the number of deaths for viral hepatitis and sequelae of viral hepatitis from 2001 to 2010

The data in the first file will be used as normalization factor for the number of deaths for viral hepatitis. Therefore, the data that will be analyzed is the ratio of deaths for viral hepatitis over the total number of deaths in that country.
Our objective for this analysis is to predict for each country the number of deaths for viral hepatitis in the following years. We decided not to make any prediction for a country outside the list because, even if there could be a hyperdistribution which the parameters of our models could follow, we considered that there could be too many differences among the countries which would make such a prediction unreliable.

## Model description
As first choice we will assess the use of a Separate model. We will consider each country as a separate group, and so we will have a distribution that describe each of them independently. 

![Separate Model](./separate.png)

We will also evaluate a Hierarchial model, in order to verify that our initial assumption about the independece of the distribution of each country is correct. In other words, we expect that the hierarchial model will perform worse than the Separate model, which is our first choice.

![Hierarchial Model](./hierarchial.png)

## Prior choices
Our prior hypothesis are the followings:
 - we assume that the data for each  distributed $y_{ij}\mid\theta_j \sim \mathcal{N}(\mu_ {ij}, \sigma_j)$
 - as prior distribution we will use an uninformative flat prior $\theta_j \sim \mathcal{U}([0,1])=Beta(1,1)$

In the Separate model we will fit a linear gaussian model for each group independently:
$$\mu_{ij} = \alpha_j + \beta_j x_{ij}$$
and then for the Hierarchial model we will add the layer represented by the hyperdistributions for $\alpha$, $\beta$ and $\sigma$:
$$\alpha \sim \mathcal{N}(\mu_\alpha, \sigma_\alpha)$$
$$\beta \sim \mathcal{N}(\mu_\beta, \sigma_\beta)$$
$$\sigma \sim Inv-\chi^2(\sigma^2_0, \nu_0)$$


In [1]:
import pystan
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import stan_utility
import matplotlib as mpl

d = pd.read_csv("../dataset/deads.txt", sep=" ", header=None, skiprows=1)
h = pd.read_csv("../dataset/hepatitis.txt", sep=" ", header=None, skiprows=1)
countries = d[0].as_matrix()
d = d.iloc[:, 1:d.shape[1]].as_matrix()
h = h.iloc[:, 1:h.shape[1]].as_matrix()

data = h/d

# Separate model

In [6]:
separate_model_code = '''
data {
    int<lower=0> N;
    vector[N] x; // group indicator
    vector[N] y;
    real xpred;
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}

transformed parameters {
  vector[N] mu;
  mu = alpha + beta*x;
}

model {
    y ~ normal(mu, sigma);
}

generated quantities {
    real ypred;
    ypred = normal_rng(alpha + beta*xpred, sigma);
}
'''

sm = stan_utility.compile_model_plus(separate_model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_51e32618796eea42cd911538cb8878a4 NOW.


In [7]:
N = 10
x = range(2001, 2011)
xpred=2011
K = 25 

samples = []
for i in range(K):
    y = np.log(data[i]).ravel()   # observations
    separate_model_data = dict(
        N = N,
        #K = K,  # 25 contries
        x = x,  # group indicators
        y = y,  # observations
        xpred=xpred
    )

    samples.append(sm.sampling(n_jobs=4, data=separate_model_data))

    print('Completed ', i)

Completed  0
Completed  1
Completed  2
Completed  3
Completed  4
Completed  5
Completed  6
Completed  7
Completed  8
Completed  9
Completed  10
Completed  11
Completed  12
Completed  13
Completed  14
Completed  15
Completed  16
Completed  17
Completed  18
Completed  19
Completed  20
Completed  21
Completed  22
Completed  23
Completed  24


In [8]:
for convergence in samples:
    print(convergence)

Inference for Stan model: anon_model_51e32618796eea42cd911538cb8878a4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha  72.33    6.43  55.28 -34.69  36.94   71.4 105.72 193.98     74   1.05
beta   -0.04  3.2e-3   0.03   -0.1  -0.06  -0.04  -0.02   0.01     74   1.05
sigma   0.24  3.6e-3   0.06   0.15    0.2   0.23   0.27   0.41    307   1.02
mu[0]   -7.7    0.02   0.15  -7.99  -7.79   -7.7  -7.61  -7.39     90   1.04
mu[1]  -7.74    0.01   0.12  -7.99  -7.82  -7.74  -7.66  -7.48    102   1.03
mu[2]  -7.78  8.7e-3    0.1  -7.99  -7.84  -7.78  -7.72  -7.56    145   1.03
mu[3]  -7.82  5.3e-3   0.09   -8.0  -7.88  -7.82  -7.77  -7.63    282   1.01
mu[4]  -7.86  1.5e-3   0.08  -8.02  -7.91  -7.86  -7.81   -7.7   2659    1.0
mu[5]   -7.9  1.3e-3   0.08  -8.07  -7.95   -7.9  -7.85  -7.74   3616    1.0
mu[6]  -7.94  3.6e-3   0.09  -8.12   -8.0

Inference for Stan model: anon_model_51e32618796eea42cd911538cb8878a4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha -18.52    4.44   40.9 -105.3 -44.07 -17.37   8.46  57.13     85   1.02
beta  5.7e-3  2.2e-3   0.02  -0.03-7.8e-3 5.1e-3   0.02   0.05     85   1.02
sigma   0.18  5.7e-3   0.06    0.1   0.14   0.17    0.2   0.34    105   1.05
mu[0]  -7.19  9.8e-3   0.11  -7.42  -7.25  -7.19  -7.12  -6.98    124   1.02
mu[1]  -7.18  7.6e-3   0.09  -7.38  -7.23  -7.18  -7.12   -7.0    149   1.02
mu[2]  -7.18  5.4e-3   0.08  -7.35  -7.22  -7.18  -7.13  -7.02    209   1.01
mu[3]  -7.17  3.2e-3   0.07  -7.32  -7.21  -7.17  -7.13  -7.04    435   1.01
mu[4]  -7.16  1.1e-3   0.06   -7.3   -7.2  -7.16  -7.13  -7.05   3311    1.0
mu[5]  -7.16  1.3e-3   0.06  -7.29  -7.19  -7.16  -7.12  -7.04   2226    1.0
mu[6]  -7.15  3.6e-3   0.07  -7.29  -7.19

Inference for Stan model: anon_model_51e32618796eea42cd911538cb8878a4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha  32.09    4.12  45.29 -67.17   6.07  30.88  57.51 128.36    121   1.02
beta   -0.02  2.1e-3   0.02  -0.07  -0.03  -0.02-7.0e-3   0.03    121   1.02
sigma   0.19  3.8e-3   0.06   0.11   0.14   0.17   0.21   0.34    253   1.01
mu[0]  -7.76  9.3e-3   0.12  -8.01  -7.83  -7.76  -7.69  -7.51    164   1.01
mu[1]  -7.78  7.3e-3    0.1  -7.99  -7.84  -7.78  -7.72  -7.57    191   1.01
mu[2]   -7.8  5.3e-3   0.08  -7.97  -7.85   -7.8  -7.75  -7.63    255   1.01
mu[3]  -7.82  3.3e-3   0.07  -7.96  -7.86  -7.82  -7.78  -7.68    466    1.0
mu[4]  -7.84  1.1e-3   0.06  -7.96  -7.88  -7.84   -7.8  -7.72   3339    1.0
mu[5]  -7.86  1.1e-3   0.06  -7.98   -7.9  -7.86  -7.82  -7.74   3469    1.0
mu[6]  -7.88  3.3e-3   0.07  -8.01  -7.92

Inference for Stan model: anon_model_51e32618796eea42cd911538cb8878a4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha -27.96   15.06 142.08 -317.1 -114.8 -28.36  56.33 271.51     89   1.05
beta    0.01  7.5e-3   0.07  -0.14  -0.03   0.01   0.05   0.15     89   1.05
sigma   0.66    0.01    0.2   0.39   0.52   0.62   0.76   1.14    271   1.01
mu[0]   -7.8    0.03   0.39  -8.58  -8.04   -7.8  -7.55  -7.01    208   1.03
mu[1]  -7.79    0.02   0.33  -8.48  -7.99  -7.79  -7.58  -7.11    248   1.03
mu[2]  -7.78    0.02   0.28  -8.36  -7.95  -7.78   -7.6  -7.21    353   1.02
mu[3]  -7.77  9.7e-3   0.25  -8.27  -7.91  -7.77  -7.62  -7.27    641   1.01
mu[4]  -7.76  3.7e-3   0.22  -8.22  -7.89  -7.76  -7.62  -7.31   3770    1.0
mu[5]  -7.75  3.6e-3   0.22  -8.21  -7.88  -7.74  -7.61  -7.29   3806    1.0
mu[6]  -7.74  9.0e-3   0.24  -8.23  -7.88

Inference for Stan model: anon_model_51e32618796eea42cd911538cb8878a4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha 260.13   16.63 171.97 -62.51 138.83 260.04 372.58 600.19    107   1.04
beta   -0.13  8.3e-3   0.09   -0.3  -0.19  -0.13  -0.07   0.03    107   1.04
sigma   0.84    0.02   0.23   0.52   0.69    0.8   0.94   1.44    192   1.02
mu[0]  -7.57    0.04   0.47  -8.46  -7.89  -7.59  -7.27  -6.62    159   1.02
mu[1]  -7.71    0.03   0.41  -8.48  -7.98  -7.72  -7.45  -6.88    193   1.02
mu[2]  -7.84    0.02   0.35  -8.52  -8.06  -7.85  -7.62  -7.13    275   1.01
mu[3]  -7.97    0.01    0.3  -8.57  -8.16  -7.98   -7.8  -7.33    718   1.01
mu[4]  -8.11  5.1e-3   0.28  -8.66  -8.28  -8.11  -7.94  -7.54   2989    1.0
mu[5]  -8.24  5.4e-3   0.28   -8.8  -8.41  -8.24  -8.07  -7.67   2657    1.0
mu[6]  -8.38    0.01    0.3  -8.97  -8.57

# Hierarchical

In [12]:
hierarchical_model_code = '''
data {
  int<lower=0> N; // number of data points
  int<lower=0> K; // number of groups
  int<lower=1,upper=K> x[N]; // group indicator
  vector[N] y; //
}
parameters {
  vector[K] mu;        // group means
  real<lower=0> sigma; // common std
}
model {
  y ~ normal(mu[x], sigma);
}
'''

sm_hierarchical = stan_utility.compile_model_plus(hierarchical_model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_dabc85ba871929f691cd2c541e749fce NOW.


In [13]:
K=2
nj = 10
N = nj*K
x = np.array([i for i in range(1,K+1) for j in range(10)])
# print(x)
y = np.log(data[0:K]).ravel()  # observations
xpred=K+1

hierarchical_model_data = dict(
    N = N,
    K = K,  # 25 contries
    x = x,  # group indicators
    y = y,  # observations
    xpred=xpred
)

samples_hierarchical = sm_hierarchical.sampling(n_jobs=1, data=hierarchical_model_data)

print('Completed ', samples_hierarchical)

Completed  Inference for Stan model: anon_model_dabc85ba871929f691cd2c541e749fce.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu[0]  -7.88  1.9e-3   0.11  -8.09  -7.94  -7.88  -7.81  -7.66   3229    1.0
mu[1]  -8.47  1.9e-3    0.1  -8.68  -8.54  -8.47   -8.4  -8.26   3030    1.0
sigma   0.33  1.2e-3   0.06   0.23   0.29   0.32   0.36   0.47   2535    1.0
lp__   11.96    0.04   1.38   8.66  11.37  12.28  12.93   13.5   1463    1.0

Samples were drawn using NUTS at Sun Dec  3 19:48:06 2017.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


## How Stan model is run

### Separate
Given that we are using a separate model, we created a Stan model which is run once on the data of each country. This gives us K predictions, one for each analysed country.

### Hierarchical
In the hierarchical model we created a single Stan model to analyse the whole dataset at one, considering each parameter `alpha` (respectively `beta`) to be following a common hyperdistribution across all the countries. 

In [ ]:
plt.hist(np.exp(samples['ypred']), 50)
plt.xlabel('y-prediction for x={}'.format(2011))
plt.show()

In [ ]:

color_scatter = 'C0'  # 'C0' for default color #0
color_line = 'C1'     # 'C1' for default color #1

color_shade = (
    1 - 0.1*(1 - np.array(mpl.colors.to_rgb(color_line)))
)

plt.fill_between(
    x[0],
    np.percentile(samples['mu'], 5, axis=0),
    np.percentile(samples['mu'], 95, axis=0),
    color=color_shade
)

plt.plot(
    x[0],
    np.percentile(samples['mu'], 50, axis=0),
    color=color_line,
    linewidth=1
)

plt.scatter(x[0], y, 5, color=color_scatter)
plt.xlim((2001, 2010))
plt.tight_layout()
plt.show()